<a href="https://colab.research.google.com/github/JapiKredi/RAG_GCP_GEMINI_VERTEXAI/blob/main/RAG_GCP_GEMINI_VERTEXAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --user google-cloud-aiplatform

In [2]:
!pip install ragstack-ai

In [3]:
from google.colab import userdata
PROJECT_ID = userdata.get('GCP_PROJECT_ID')

In [4]:
from google.colab import userdata
LOCATION = userdata.get('GCP_LOCATION')

In [5]:
import os

In [6]:
os.environ['GCP_PROJECT_ID'] = PROJECT_ID

In [7]:
os.environ['LOCATION'] = LOCATION

In [8]:
!gcloud config set project {os.getenv('GCP_PROJECT_ID')}

Updated property [core/project].


In [9]:
import sys

if "google.colab" in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

In [10]:
import vertexai

In [11]:
!gcloud auth list

       Credentialed Accounts
ACTIVE  ACCOUNT
*       jasperbongers1974@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [12]:
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [13]:
from vertexai.preview.generative_models import GenerativeModel

In [14]:
model = GenerativeModel('gemini-1.0-pro')

In [15]:
response = model.generate_content('what is the colour of the sky')

In [16]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "The colour of the sky is blue. This is because the Earth\'s atmosphere scatters sunlight in all directions. Blue light is scattered more than other colours, which is why the sky appears blue.\n\nAt sunrise and sunset, the sky can appear red or orange. This is because the sunlight has to travel through more of the Earth\'s atmosphere to reach our eyes. This causes the blue light to be scattered even more, and the red and orange light to be scattered less. This is why the sky appears red or orange at sunrise and sunset."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.05665242671966553
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.04193052276968956
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.08555634319782257
    severity: HARM_SEVERIT

# RAG System

In [22]:
ASTRA_DB_API_ENDPOINT="https://fded6382-857e-4eab-83f2-ceca736c0712-us-east-2.apps.astra.datastax.com"
ASTRA_DB_APPLICATION_TOKEN="AstraCS:qfKCQxsikjlrQZpunuiubGKg:8c0059b84e27a87378eb2a49e4d38f3bfcb0de70b66c23cfceafbb69a379e22f"

In [23]:
os.environ["ASTRA_DB_ENDPOINT"]=ASTRA_DB_API_ENDPOINT

In [24]:
os.environ["ASTRA_DB_TOKEN"]=ASTRA_DB_APPLICATION_TOKEN

In [25]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part
)

In [26]:
prompt = """Create a numbered list of 10 items. Each item in the list should be a trend in the tech industry.

Each trend should be less than 5 words."""

In [27]:
response=model.generate_content(prompt, stream=True)

In [28]:
for response in response:
    print(response.text, end="")

## Top 10 Tech Industry Trends:

1. Artificial Intelligence Everywhere
2. The Rise of the Metaverse
3. Quantum Computing Leaps Forward
4. Blockchain Beyond Crypto
5. Personalized Healthcare via Tech
6. Hyper Automation Takes Over
7. Cybersecurity in the Spotlight
8. Sustainability Drives Innovation
9. The Creator Economy Booms
10. Decentralization Disrupts Industries

In [31]:
generation_config =GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

In [32]:
response=model.generate_content(prompt,generation_config=generation_config, stream=True)

In [33]:
for response in response:
    print(response.text, end="")

## Top 10 Tech Trends for 2023:

1.  AI Personalization
2.  Cybersecurity Mesh
3.  Generative AI
4.  Sustainable Tech
5.  Hyperautomation
6.  Quantum Computing
7.  Decentralized Finance
8.  Metaverse Expansion
9.  Low-Code Development
10. Digital Wellbeing

In [34]:
!pip install requests

In [35]:
import requests

In [36]:
source_img_data =requests.get("https://drive.google.com/uc?export=view&id=15ddcn-AIxpvRdWcFGvIr77XLWdo4Maof").content

In [37]:
with open('coffee_maker_part.png', 'wb') as handler:
  handler.write(source_img_data)

In [38]:
from langchain_google_vertexai import ChatVertexAI

In [39]:
from langchain.schema.messages import HumanMessage
from PIL import Image, ImageFile
import os, sys

In [40]:
chat = ChatVertexAI(model_name="gemini-1.0-pro-vision")

In [41]:
image_message = {
    "type": "image_url",
    "image_url": {"url": "coffee_maker_part.png"},
}
text_message = {
    "type": "text",
    "text": "What is this image? Share a link to purchase a replacement",
}

In [42]:
message = HumanMessage(content=[text_message, image_message])

In [43]:
message

HumanMessage(content=[{'type': 'text', 'text': 'What is this image? Share a link to purchase a replacement'}, {'type': 'image_url', 'image_url': {'url': 'coffee_maker_part.png'}}])

In [44]:
print(chat([message]).content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 This is a replacement basket for a Breville Duo Temp Pro espresso machine.

https://www.amazon.com/dp/B07Q49X69G?ref=myi_title_dp


RAG: I can pass the image and based on that i can take a recommendation from my model.

In [46]:
import pandas as pd

In [47]:
d = {'name': ["Saucer", "Saucer Ceramic", "Milk Jug Assembly", "Handle Steam Wand Kit (New Version From 0735 PDC)", "Spout Juice Small (From 0637 to 1041 PDC)", "Cleaning Steam Wand", "Jug Frothing", "Spoon Tamping 50mm", "Collar Grouphead 50mm", "Filter 2 Cup Dual Wall 50mm", "Filter 1 Cup 50mm", "Water Tank Assembly", "Portafilter Assembly 50mm", "Milk Jug Assembly", "Filter 2 Cup 50mm" ],
     'url': ["https://www.breville.com/us/en/parts-accessories/parts/sp0014946.html?sku=SP0014946", "https://www.breville.com/us/en/parts-accessories/parts/sp0014914.html?sku=SP0014914", "https://www.breville.com/us/en/parts-accessories/parts/sp0011391.html?sku=SP0011391", "https://www.breville.com/us/en/parts-accessories/parts/sp0010719.html?sku=SP0010719", "https://www.breville.com/us/en/parts-accessories/parts/sp0010718.html?sku=SP0010718", "https://www.breville.com/us/en/parts-accessories/parts/sp0003247.html?sku=SP0003247", "https://www.breville.com/us/en/parts-accessories/parts/sp0003246.html?sku=SP0003246", "https://www.breville.com/us/en/parts-accessories/parts/sp0003243.html?sku=SP0003243", "https://www.breville.com/us/en/parts-accessories/parts/sp0003232.html?sku=SP0003232", "https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231", "https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230", "https://www.breville.com/us/en/parts-accessories/parts/sp0003225.html?sku=SP0003225", "https://www.breville.com/us/en/parts-accessories/parts/sp0003216.html?sku=SP0003216", "https://www.breville.com/us/en/parts-accessories/parts/sp0001875.html?sku=SP0001875", "https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166"],
     'price': ["10.95", "4.99", "14.95", "8.95", "10.95", "6.95", "24.95", "8.95", "6.95", "12.95", "12.95", "14.95", "10.95", "16.95", "11.95"],
     'image': ["https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0014946/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0014914/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0011391/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0010719/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0010718/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003247/tile.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/BES250/SP0003246/SP0003246_IMAGE1_400X400.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/ESP8/SP0003243/SP0003243_IMAGE1_400X400.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/ESP8/SP0003232/SP0003232_IMAGE1_400x400.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003225/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003216/tile.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0001875/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0000166/tile.jpg"]}

In [48]:
df = pd.DataFrame(data=d)
df

,name,url,price,image
0,Saucer,https://www.breville.com/us/en/parts-accessori...,10.95,https://www.breville.com/content/dam/breville/...
1,Saucer Ceramic,https://www.breville.com/us/en/parts-accessori...,4.99,https://www.breville.com/content/dam/breville/...
2,Milk Jug Assembly,https://www.breville.com/us/en/parts-accessori...,14.95,https://www.breville.com/content/dam/breville/...
3,Handle Steam Wand Kit (New Version From 0735 PDC),https://www.breville.com/us/en/parts-accessori...,8.95,https://www.breville.com/content/dam/breville/...
4,Spout Juice Small (From 0637 to 1041 PDC),https://www.breville.com/us/en/parts-accessori...,10.95,https://www.breville.com/content/dam/breville/...
5,Cleaning Steam Wand,https://www.breville.com/us/en/parts-accessori...,6.95,https://www.breville.com/content/dam/breville/...
6,Jug Frothing,https://www.breville.com/us/en/parts-accessori...,24.95,https://assets.breville.com/cdn-cgi/image/widt...
7,Spoon Tamping 50mm,https://www.breville.com/us/en/parts-accessori...,8.95,https://assets.breville.com/cdn-cgi/image/widt...
8,Collar Grouphead 50mm,https://www.breville.com/us/en/parts-accessori...,6.95,https://assets.breville.com/cdn-cgi/image/widt...
9,Filter 2 Cup Dual Wall 50mm,https://www.breville.com/us/en/parts-accessori...,12.95,https://www.breville.com/content/dam/breville/...


In [49]:
df.head()

,name,url,price,image
0,Saucer,https://www.breville.com/us/en/parts-accessori...,10.95,https://www.breville.com/content/dam/breville/...
1,Saucer Ceramic,https://www.breville.com/us/en/parts-accessori...,4.99,https://www.breville.com/content/dam/breville/...
2,Milk Jug Assembly,https://www.breville.com/us/en/parts-accessori...,14.95,https://www.breville.com/content/dam/breville/...
3,Handle Steam Wand Kit (New Version From 0735 PDC),https://www.breville.com/us/en/parts-accessori...,8.95,https://www.breville.com/content/dam/breville/...
4,Spout Juice Small (From 0637 to 1041 PDC),https://www.breville.com/us/en/parts-accessori...,10.95,https://www.breville.com/content/dam/breville/...


In [50]:
df["url"]

0     https://www.breville.com/us/en/parts-accessori...
1     https://www.breville.com/us/en/parts-accessori...
2     https://www.breville.com/us/en/parts-accessori...
3     https://www.breville.com/us/en/parts-accessori...
4     https://www.breville.com/us/en/parts-accessori...
5     https://www.breville.com/us/en/parts-accessori...
6     https://www.breville.com/us/en/parts-accessori...
7     https://www.breville.com/us/en/parts-accessori...
8     https://www.breville.com/us/en/parts-accessori...
9     https://www.breville.com/us/en/parts-accessori...
10    https://www.breville.com/us/en/parts-accessori...
11    https://www.breville.com/us/en/parts-accessori...
12    https://www.breville.com/us/en/parts-accessori...
13    https://www.breville.com/us/en/parts-accessori...
14    https://www.breville.com/us/en/parts-accessori...
Name: url, dtype: object

In [51]:
df["image"]

0     https://www.breville.com/content/dam/breville/...
1     https://www.breville.com/content/dam/breville/...
2     https://www.breville.com/content/dam/breville/...
3     https://www.breville.com/content/dam/breville/...
4     https://www.breville.com/content/dam/breville/...
5     https://www.breville.com/content/dam/breville/...
6     https://assets.breville.com/cdn-cgi/image/widt...
7     https://assets.breville.com/cdn-cgi/image/widt...
8     https://assets.breville.com/cdn-cgi/image/widt...
9     https://www.breville.com/content/dam/breville/...
10    https://www.breville.com/content/dam/breville/...
11    https://www.breville.com/content/dam/breville/...
12    https://www.breville.com/content/dam/breville/...
13    https://www.breville.com/content/dam/breville/...
14    https://www.breville.com/content/dam/breville/...
Name: image, dtype: object

In [52]:
import vertexai, json, requests